# Meta Kaggle Overview - Datasets
Kaggle's public data on competitions, users, submission scores, and kernels

- https://www.kaggle.com/datasets/kaggle/meta-kaggle


In [12]:
%%html
<style type='text/css'>
.CodeMirror {
    font-size: 14px; 
    font-family: 'Droid Sans Mono';
}
</style>

In [13]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [14]:
import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, BooleanType
from pyspark.sql.functions import *

In [16]:
spark = (SparkSession
        .builder
        .appName("meta-kaggle-data-analysis")
        .config("spark.driver.memory", "4g")
        .config("spark.executor.instances", 4)
        .config("spark.executor.memory", "1g")
        .config("spark.executor.cores", 2)
        .getOrCreate()
        )

In [17]:
spark

****

## Datasets

- Here, I am not using the original version of the `DatasetVersions.csv` file.
- I have formatted the original a bit.
- The formatted file will have:
    - Summary attribute
    - All new lines in "Description" are replaced with three tildas (~~~)
    
- Field separator is Unit Separator char (ASCII - 31).

In [18]:
data_files_path = "/home/rk/Desktop/data/kaggle-meta/formatted"

****

## Datasets

In [19]:
datasets_file = f"{data_files_path}/DatasetVersions_fmt_1.csv"

In [20]:
schema = StructType([
    StructField("Id", IntegerType()),
    StructField("DatasetId", IntegerType()),
    StructField("DatasourceVersionId", IntegerType()),
    StructField("CreatorUserId", IntegerType()),
    StructField("LicenseName", StringType()),
    StructField("CreationDate", StringType()),
    StructField("VersionNumber", IntegerType()),
    StructField("Title", StringType()),
    StructField("Slug", StringType()),
    StructField("Subtitle", StringType()),
    StructField("Summary", StringType()),
    StructField("Description", StringType()),
])

In [21]:
datasets = (
    spark.read.format("csv")
    .option("header", False)
    .option("sep", chr(31))
    .schema(schema)
    .load(datasets_file)
) 

datasets.createOrReplaceTempView("datasets")
datasets.printSchema()

root
 |-- Id: integer (nullable = true)
 |-- DatasetId: integer (nullable = true)
 |-- DatasourceVersionId: integer (nullable = true)
 |-- CreatorUserId: integer (nullable = true)
 |-- LicenseName: string (nullable = true)
 |-- CreationDate: string (nullable = true)
 |-- VersionNumber: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Slug: string (nullable = true)
 |-- Subtitle: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Description: string (nullable = true)



****

In [22]:
spark.sql("""
SELECT
    Id
  , DatasetId
  , CreatorUserId
  , LicenseName
  , CreationDate
  , Title
  , Subtitle
  , Summary
  , REPLACE(Description, '~~~', '\n') AS Description
FROM 
    datasets 
limit 2
""").toPandas()

,Id,DatasetId,CreatorUserId,LicenseName,CreationDate,Title,Subtitle,Summary,Description
0,6,6,1,CC0: Public Domain,07/18/2015 00:51:12,2013 American Community Survey,Find insights in the 2013 American Community Survey,The [American Community Survey](http://www.census.gov/programs-surveys/acs/) is an ongoing survey from the US Census Bureau. In this survey approximately 3.5 million households per year are asked detailed questions about who they are and how they live. Many [topics](http://www.census.gov/programs-surveys/acs/guidance/subjects.html) are covered including ancestry education work transportation internet use and residency.,"The [American Community Survey](http://www.census.gov/programs-surveys/acs/) is an ongoing survey from the US Census Bureau. In this survey approximately 3.5 million households per year are asked detailed questions about who they are and how they live. Many [topics](http://www.census.gov/programs-surveys/acs/guidance/subjects.html) are covered including ancestry education work transportation internet use and residency.\n\nThe responses reveal a fascinating, granular snapshot into the lives of many Americans.\n\n[![Number of Households Surveyed](https://www.kaggle.io/svf/30288/6679caba9ecb435abbb518d96ca1fcd3/number_of_households_surveyed.png)](https://www.kaggle.com/benhamner/d/census/2013-american-community-survey/number-of-households-surveyed-map)\n\nWe''re publishing this data on scripts to make it easy for you to explore this rich dataset, share your work, and collaborate with other data scientists. No data download or local environment needed! We''ve also added shapefil..."
1,7,7,1,Reddit API Terms,08/04/2015 23:59:00,May 2015 Reddit Comments,Get personal with a dataset of comments from May 2015,Recently Reddit released [an enormous dataset](https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment/) containing all ~1.7 billion of their publicly available comments. The full dataset is an unwieldy 1+ terabyte uncompressed so we've decided to host a small portion of the comments here for Kagglers to explore. (You don't even need to leave your browser!),"Recently Reddit released [an enormous dataset](https://www.reddit.com/r/datasets/comments/3bxlg7/i_have_every_publicly_available_reddit_comment/) containing all ~1.7 billion of their publicly available comments. The full dataset is an unwieldy 1+ terabyte uncompressed so we've decided to host a small portion of the comments here for Kagglers to explore. (You don't even need to leave your browser!)\n\nYou can find all the comments from May 2015 on scripts for your natural language processing pleasure. What had redditors laughing, bickering, and NSFW-ing this spring?\n\nWho knows? Top visualizations may just end up on [Reddit](https://www.reddit.com/r/dataisbeautiful).\n\n## Data Description\n\nThe database has one table, `May2015`, with the following fields:\n\n - created_utc\n - ups\n - subreddit_id\n - link_id\n - name\n - score_hidden\n - author_flair_css_class\n - author_flair_text\n - subreddit\n - id\n - removal_reason\n - gilded\n - downs\n - archived\n - author\n - score\n ..."


In [23]:
spark.sql("SELECT COUNT(*) FROM datasets").toPandas()

,count(1)
0,1046579


### Users

In [24]:
users_file = "/home/rk/Desktop/data/kaggle-meta/Users.csv"
users = spark.read.csv(users_file, header=True)
users.createOrReplaceTempView("users")
users.printSchema()

root
 |-- Id: string (nullable = true)
 |-- UserName: string (nullable = true)
 |-- DisplayName: string (nullable = true)
 |-- RegisterDate: string (nullable = true)
 |-- PerformanceTier: string (nullable = true)



In [25]:
datasets_out = spark.sql("""
SELECT
    u.Id AS UserId
  , u.UserName
  , u.DisplayName
  , u.PerformanceTier
  , CONCAT("https://www.kaggle.com/datasets/", u.UserName, "/", a.Slug) AS dataset_url
  , a.DatasetId
  , a.LicenseName
  , CASE
      WHEN LENGTH(a.CreationDate) >= 10 AND 
           regexp_extract(a.CreationDate, r"(\d{2}/\d{2}/\d{4})", 1) != "" AND
           regexp_extract(a.CreationDate, r"(\d{2}/\d{2}/\d{4})", 1) IS NOT NULL THEN
          to_date(SUBSTR(a.CreationDate, 1, 10), "MM/dd/yyyy")
      ELSE
          '2010-01-01'
    END AS CreationDate
  , a.Title
  , a.Subtitle
  , a.Summary  
FROM
    datasets a
  , users u
WHERE
    a.CreatorUserId = u.Id
""")

datasets_out.limit(3).toPandas()

,UserId,UserName,DisplayName,PerformanceTier,dataset_url,DatasetId,LicenseName,CreationDate,Title,Subtitle,Summary
0,808,alexxanderlarko,Alexander Larko,4,https://www.kaggle.com/datasets/alexxanderlarko/melanoma384x384ext,752404,Unknown,2020-07-02,melanoma-384x384ext,None,Initial release 0 0
1,808,alexxanderlarko,Alexander Larko,4,https://www.kaggle.com/datasets/alexxanderlarko/melanoma384x384ext,752404,Unknown,2020-07-02,melanoma-384x384ext,None,melanoma-384x384ext 0 0
2,808,alexxanderlarko,Alexander Larko,4,https://www.kaggle.com/datasets/alexxanderlarko/data384,831541,Unknown,2020-08-15,data384,None,Initial release 0 0


### Write to a CSV file

In [26]:
out_file = f"{data_files_path}/DatasetVersions_fmt.csv"
datasets_out.orderBy("CreationDate", ascending=False).toPandas().to_csv(out_file, index=False)

In [28]:
# spark.stop()